# Example usage of halo_trace 

First, to import the relevant packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import halo_trace as ht
import os
import pandas as pd

%matplotlib inline

A bit about the tracing code:

The tracing code is fairly slow, but should be robust. By default it will cross check steps, meaning it calculates the main progenitor both 1 and 2 steps apart, to make sure they match. If it loses a main progenitor, it can also pick it up again in some cases. Because this process is so slow, usually what you want to do is run it once and save the output, then read in the output for all subsequent uses. Main progenitors are identified by maximizing a merit function (as in AHF's merger tree code), where the merit function of halo

\begin{equation}
\max_j\left(M_{ij} = \frac{N_{i \cap j}}{N_i N_j}\right),
\end{equation}

where $N_i$ and $N_j$ are the number of particles in halos in steps $i$ and $j$, N$_{i \cap j}$ is the number of shared particles. The merit function is maximized in both directions (step 1 to 2 and step 2 to 1), and a match is recorded only if the identified halos match.

Anyway...

The code traces halos and returns a pandas dataframe identifying the main progenitor at each step.

The only mandatory argument in the trace is `sim_base`, which is the path to the directory holding all your simulation time steps. You can specify a file name for the saved trace using `save_file`. You can trace specific halos by spcifying `grplist` (e.g., `grplist=[1, 3, 6]` will trace halos 1, 3, and 6 back), or you can trace all halos above some particle threshold. For all arguments, you can run `ht.tracing.trace_halos?`. So, if want to trace all halos with more than 1000 particles at z=0, no earlier than step 400, then this would run the trace (replacing `sim_base` with your own path):

In [2]:
sim_base = '/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH/'
trace = ht.tracing.trace_halos(sim_base=sim_base, maxstep=400, min_ntot=500)

h148.cosmo50PLK.6144g3HbwK1BH.000768 does not exist or has incorrect permissions
File exists: /data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50PLK.6144g3HbwK1BH.004096/h148.cosmo50PLK.6144g3HbwK1BH.004096.trace_back.hdf5
Aborting


By default, it will save your trace. In my case I've already run this trace, so it will decline to overwrite it. In your case, you'll see information printed to screen as the trace goes on. Since I've already created the trace, I'll go ahead and load it now:

In [3]:
trace = pd.read_hdf(sim_base + 
                    'h148.cosmo50PLK.6144g3HbwK1BH.004096/h148.cosmo50PLK.6144g3HbwK1BH.004096.trace_back.hdf5')

The index is the present day halo ID. Each column represents the main progenitor of a halo at that timestep:

In [4]:
trace.head()

,004032,003936,003840,003810,003760,003744,003671,003648,003606,003552,...,000512,000457,000384,000347,000275,000256,000225,000188,000139,000128
004096,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,85.0,179.0,183.0,159.0,140.0,176.0,186
2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,5.0,5.0,11.0,23.0,28.0,25.0,19.0,19.0,20.0,30
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,12.0,12.0,12.0,14.0,14.0,12.0,9.0,5.0,2.0,1
4,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,8.0,7.0,6.0,5.0,3.0,3.0,4.0,4.0,3.0,9
5,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,14.0,18.0,15.0,13.0,52.0,55.0,50.0,44.0,39.0,41


If you don't know how to use pandas, this might take a bit to get used to. To see the main progenitor IDs of halo 7, for example, you do this:

In [5]:
trace.loc[7]

004032     7.0
003936     7.0
003840     7.0
003810     7.0
003760     7.0
003744     7.0
003671     7.0
003648     7.0
003606     7.0
003552     7.0
003456     8.0
003360     8.0
003264     9.0
003195     9.0
003168     9.0
003072     8.0
002976     9.0
002880     9.0
002784     9.0
002688     9.0
002592     9.0
002555     9.0
002496     9.0
002400     9.0
002304     9.0
002208     9.0
002112     9.0
002088     9.0
002048     9.0
001920     9.0
001740     9.0
001664     9.0
001536    11.0
001476    10.0
001408    11.0
001280    15.0
001270    16.0
001152    17.0
001106    18.0
001024    20.0
000777    28.0
000640    35.0
000637    36.0
000512    28.0
000457    31.0
000384    34.0
000347    35.0
000275    27.0
000256    24.0
000225    18.0
000188    14.0
000139    11.0
000128      13
Name: 7, dtype: object

To see the main progenitor ID just at step 000188, for example, you would do this:

In [6]:
trace.loc[7, '000188']

14.0

or for several steps:

In [7]:
trace.loc[7, ['000139', '000188']]

000139    11.0
000188    14.0
Name: 7, dtype: object

The -1s (and, in some cases, NaNs) represent steps where the main progenitor could not be idenified. Most larger halos are traced back pretty far. here is an example of a halo that wasn't:

In [8]:
trace.loc[4890]

004032    4843.0
003936    4894.0
003840    4945.0
003810    4915.0
003760    5101.0
003744    4900.0
003671    4871.0
003648    4869.0
003606    4843.0
003552    4804.0
003456    4652.0
003360    4525.0
003264    4403.0
003195    3537.0
003168    3448.0
003072    3368.0
002976    3388.0
002880    3408.0
002784    3436.0
002688    3501.0
002592    3544.0
002555    3539.0
002496    3518.0
002400    3416.0
002304    3372.0
002208    3342.0
002112    3198.0
002088    3142.0
002048    2818.0
001920    2499.0
001740    2511.0
001664    2662.0
001536    2518.0
001476    2513.0
001408    2533.0
001280    2686.0
001270    2675.0
001152    2703.0
001106    2749.0
001024    2809.0
000777    3444.0
000640    3829.0
000637    3888.0
000512    3905.0
000457    4209.0
000384    4980.0
000347    6181.0
000275    7550.0
000256    7360.0
000225    7616.0
000188    9724.0
000139      -1.0
000128        -1
Name: 4890, dtype: object

Using the identified halos from the trace, you can read in any value in the amiga.stat or AHF_halos files and put it in a DataFrame of its own. Every time you read in an amiga.stat file you'll get some output information; this mainly has to do with connecting AHF files to amiga files, and you should be able to ignore it.

In [9]:
mvir_trace, rvir_trace, distance = ht.tracing.trace_quantity(sim_base, trace_df=trace, 
                                       quantity=['Mvir(M_sol)', 'Rvir(kpc)', 'Nearest'])

/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50PLK.6144g3HbwK1BH.004096/h148.cosmo50PLK.6144g3HbwK1BH.004096 

/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50PLK.6144g3HbwK1BH.004032/h148.cosmo50PLK.6144g3HbwK1BH.004032 

/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50PLK.6144g3HbwK1BH.003936/h148.cosmo50PLK.6144g3HbwK1BH.003936 

/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50PLK.6144g3HbwK1BH.003840/h148.cosmo50PLK.6144g3HbwK1BH.003840 

/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50PLK.6144g3HbwK1BH.003810/h148.cosmo50PLK.6144g3HbwK1BH.003810 

/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50PLK.6144g3HbwK1BH.003760/h148.cosmo50PLK.6144g3HbwK1BH.003760 

/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50PLK.6144g3HbwK1BH.003744/h148.cosmo50PLK.6144g3HbwK1BH.003744 

/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50

/home/applebaum/anaconda3/envs/ufds/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50PLK.6144g3HbwK1BH.001106/h148.cosmo50PLK.6144g3HbwK1BH.001106 



/home/applebaum/anaconda3/envs/ufds/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50PLK.6144g3HbwK1BH.001024/h148.cosmo50PLK.6144g3HbwK1BH.001024 



...

/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50PLK.6144g3HbwK1BH.000640/h148.cosmo50PLK.6144g3HbwK1BH.000640 



/home/applebaum/anaconda3/envs/ufds/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50PLK.6144g3HbwK1BH.000637/h148.cosmo50PLK.6144g3HbwK1BH.000637 



/home/applebaum/anaconda3/envs/ufds/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50PLK.6144g3HbwK1BH.000512/h148.cosmo50PLK.6144g3HbwK1BH.000512 

/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50PLK.6144g3HbwK1BH.000457/h148.cosmo50PLK.6144g3HbwK1BH.000457 

/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50PLK.6144g3HbwK1BH.000384/h148.cosmo50PLK.6144g3HbwK1BH.000384 

/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50PLK.6144g3HbwK1BH.000347/h148.cosmo50PLK.6144g3HbwK1BH.000347 

/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50PLK.6144g3HbwK1BH.000275/h148.cosmo50PLK.6144g3HbwK1BH.000275 

/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50PLK.6144g3HbwK1BH.000256/h148.cosmo50PLK.6144g3HbwK1BH.000256 

/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50PLK.6144g3HbwK1BH.000225/h148.cosmo50PLK.6144g3HbwK1BH.000225 

/data/REPOSITORY/e12Gals/h148.cosmo50PLK.6144g3HbwK1BH///h148.cosmo50

Again, if you don't know pandas, this might take a bit to get used to. The maximum halo mass reached by every halo throughout its history (at least, its successfully traced history) can be found in one step:

In [10]:
mvir_trace.max(axis=1)

Grp
1        2.402080e+12
2        1.327024e+11
3        6.989641e+10
4        3.851439e+10
5        4.217958e+10
             ...     
15289    2.986070e+06
15291    1.913236e+06
15292    2.574561e+06
15293    2.824864e+06
15294    3.182441e+06
Length: 11356, dtype: float64

In [11]:
mvir_trace.idxmax(axis=1)

Grp
1        004096
2        003360
3        002555
4        002592
5        002880
          ...  
15289    002400
15291    003072
15292    001024
15293    001920
15294    001664
Length: 11356, dtype: object

Or, the mass at z=6 (step 275 in this simulation):

In [12]:
mvir_trace['000275']

Grp
1        2.863878e+08
2        2.216010e+09
3        3.669721e+09
4        7.197241e+09
5        1.117208e+09
             ...     
15289             NaN
15291             NaN
15292             NaN
15293             NaN
15294             NaN
Name: 000275, Length: 11356, dtype: float64

## Troubleshooting

If you're encountering errors related to the AHF or amiga files, it's possible there are multiple versions of the AHF catalogs laying around in the directory. Make sure there is one set of AHF files, and that the amiga files correspond to that run of AHF.